In [1]:
import random
import numpy as np
import pandas as pd
import ydf
import wandb

In [ ]:
# Loop through each set of training and validation data
norm = False
subdir = 'norm' if norm else 'unnorm'
learners = []
for i in range(11):  # Assuming i ranges from 0 to 12
    # Load training and validation data
    train_file = f"covset0/{subdir}/train_{i}.csv"
    valid_file = f"covset0/{subdir}/valid_{i}.csv"

    train_df = pd.read_csv(train_file)
    valid_df = pd.read_csv(valid_file)

    # Separate features and labels
    X_train = train_df.drop(columns=['DELTA_20_QUINTILES'])  # Replace with actual label column
    y_train = train_df['DELTA_20_QUINTILES']

    X_valid = valid_df.drop(columns=['DELTA_20_QUINTILES'])  # Replace with actual label column
    y_valid = valid_df['DELTA_20_QUINTILES']

    # Combine X and y for YDF
    train_data = pd.concat([X_train, y_train], axis=1)
    valid_data = pd.concat([X_valid, y_valid], axis=1)

    # Specify the label column for YDF
    label = "DELTA_20_QUINTILES"

    # Train the Gradient Boosted Trees model
    learner = ydf.RandomForestLearner(task=ydf.Task.CLASSIFICATION,
                                    label=label).train(train_df)

    learners.append(learner)



In [ ]:
model = learners[0]

In [ ]:
dir(model)

In [ ]:
model.variable_importances()

In [ ]:
model.predict(valid_df)

In [ ]:
evaluation = learners[0].analyze(valid_df)

In [ ]:
evaluation

In [ ]:
model.evaluate(valid_data)

Combine all in sample data into train_data 
and all out of sample data into valid_data 

## Training for Random Forest begins here

In [2]:
import wandb
import ydf
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import accuracy_score

# wandb.init(project='stonks')

norm = False
learners = []
subdir = 'norm' if norm else 'unnorm'

tdf = []
vdf = []
for i in range(11):
    train_df = pd.read_csv(f'covset1/train_{i}.csv')
    valid_df = pd.read_csv(f'covset1/valid_{i}.csv')
    
    tdf.append(train_df)
    vdf.append(valid_df)

train_data = pd.concat(tdf, axis=0)
valid_data = pd.concat(vdf, axis=0)


In [7]:
train_data['TOP_5'] = train_data['DELTA_20_QUINTILES'].apply(lambda x: 1 if x == 5 else 0)
valid_data['TOP_5'] = valid_data['DELTA_20_QUINTILES'].apply(lambda x: 1 if x == 5 else 0)
train_data.drop('DELTA_20_QUINTILES', axis=1, inplace=True)
valid_data.drop('DELTA_20_QUINTILES', axis=1, inplace=True)


In [15]:
valid_data['TOP_5']

0     0
1     0
2     0
3     0
4     0
     ..
55    1
56    1
57    1
58    1
59    1
Name: TOP_5, Length: 660, dtype: int64

In [ ]:
learner.predict(valid_data.drop('TOP_5'))

In [17]:
learner.label_classes()

['0', '1']

In [22]:
valid_preds

array([0.05199997, 0.087     , 0.10600004, 0.07899998, 0.09900002,
       0.09300001, 0.06699996, 0.05599996, 0.04699998, 0.05299997,
       0.05399996, 0.05499996, 0.06599995, 0.06499995, 0.06799996,
       0.089     , 0.09      , 0.07299997, 0.06999996, 0.06799996,
       0.05999995, 0.05099997, 0.04999997, 0.05699996, 0.05099997,
       0.05099997, 0.05499996, 0.05599996, 0.07599998, 0.07199997,
       0.04599998, 0.05099997, 0.04099999, 0.05999995, 0.06999996,
       0.05399996, 0.05499996, 0.04699998, 0.04799997, 0.04699998,
       0.03899999, 0.03999999, 0.04499998, 0.03999999, 0.03799999,
       0.033     , 0.032     , 0.033     , 0.033     , 0.04099999,
       0.032     , 0.026     , 0.017     , 0.017     , 0.016     ,
       0.016     , 0.016     , 0.008     , 0.009     , 0.005     ,
       0.34099916, 0.3919985 , 0.20200023, 0.27000007, 0.3229994 ,
       0.05899996, 0.18100019, 0.08399999, 0.2979997 , 0.2839999 ,
       0.275     , 0.38099864, 0.4529977 , 0.4609976 , 0.63399

In [27]:
import numpy as np
label = 'TOP_5'

# learner = ydf.RandomForestLearner(task=ydf.Task.CLASSIFICATION, label=label, num_trees=10000,
#                                   winner_take_all=False, growing_strategy='BEST_FIRST_GLOBAL').train(train_data)

learner = (ydf.RandomForestLearner(task=ydf.Task.CLASSIFICATION,
                                        label='TOP_5',
                                        max_depth = 40,
                                        # growing_strategy='BEST_FIRST_GLOBAL',
                                        num_trees=50000).train(train_data))

valid_preds = learner.predict(valid_data.drop('TOP_5', axis=1, inplace=False))
# Assuming valid_preds contains the probabilities of class 1
threshold = 0.5
predicted_classes = (valid_preds >= threshold).astype(int)

# Now create a DataFrame for the predictions
preds = pd.DataFrame({
    'Predicted': predicted_classes,
    'Probs': valid_preds
})

# Ensure consistency in lengths and alignment
true_classes = valid_data["TOP_5"].reset_index(drop=True)
predicted_classes = preds['Predicted'].reset_index(drop=True)

# Check if lengths match
assert len(true_classes) == len(predicted_classes), "Lengths of true and predicted classes do not match."

# Calculate accuracies
total_accuracy = np.mean(true_classes == predicted_classes)

print("Accuracy: ", total_accuracy)


Train model on 2772 examples
Model trained in 0:01:04.201471
Accuracy:  0.8257575757575758


In [23]:
import numpy as np
import pandas as pd


Accuracy:  0.8212121212121212


In [11]:
valid_preds = learner.predict(valid_data)

In [12]:
valid_preds

array([0.04760023, 0.08620089, 0.093101  , 0.0751007 , 0.09820109,
       0.0868009 , 0.07400068, 0.07090063, 0.05230031, 0.05300032,
       0.05370034, 0.05320033, 0.06530053, 0.06180047, 0.07000061,
       0.08330084, 0.07660072, 0.06530053, 0.07290066, 0.07290066,
       0.06720056, 0.05920043, 0.05680039, 0.05980044, 0.05320033,
       0.05540036, 0.05820041, 0.05720039, 0.07280066, 0.06440052,
       0.0516003 , 0.05330033, 0.05130029, 0.06620055, 0.07610071,
       0.07120063, 0.07670072, 0.07050062, 0.06730057, 0.06130046,
       0.05370034, 0.05410034, 0.05670039, 0.05490036, 0.04850025,
       0.04300015, 0.03630004, 0.03910009, 0.04010011, 0.04630021,
       0.03740006, 0.03149996, 0.01989997, 0.01749998, 0.01799998,
       0.02169997, 0.01919997, 0.00939999, 0.00969999, 0.01029999,
       0.31349525, 0.35508975, 0.19120266, 0.23900346, 0.2910982 ,
       0.05420034, 0.16890228, 0.08550087, 0.2768001 , 0.26290193,
       0.25450304, 0.35488978, 0.4401785 , 0.4501772 , 0.63359

In [ ]:
valid_data



In [ ]:
preds

In [ ]:
valid_fulls = []
for i in range(11):
    valid_full = pd.read_csv(f'covset0_pred/valid_{i}_pred.csv')
    valid_fulls.append(valid_full)
    
valid_full = pd.concat(valid_fulls, axis=0)
preds.index = valid_full.index
valid_full = valid_full.loc[:, ['PCT_CHANGE_20']]
valid_full.loc[:, 'Predicted'] = preds.loc[:, 'Predicted']

In [ ]:
valid_full

In [ ]:
#TODO: 